In [ ]:
import open3d as o3d
import tensorflow as tf
import numpy as np
import os
from utilities import *
import re
import scipy
import pandas as pd
import matplotlib.pyplot as plt
import tensorflow as tf
import random
from pyntcloud import PyntCloud
from sklearn.preprocessing import PolynomialFeatures
from sklearn.linear_model import LinearRegression
from plyfile import PlyData, PlyElement
from scipy.spatial import cKDTree
from tqdm.notebook import tqdm, trange
from multiprocessing import Pool

In [ ]:
initial_path = r'write the path where you put the project'
db_path = os.path.join(initial_path, r'icip2020_perry_quality_repack_bin')
deg_metrics_path = os.path.join('data', 'icip2020_deg_metrics.json')
degraded_pcs_features_path = os.path.join('data', 'icip2020_degraded_pcs_features.csv')
degraded_pcs_features_preds_path = os.path.join('data', 'icip2020_degraded_pcs_features_preds.csv')
block_bits = 6
block_shape = [2**block_bits] * 3
bbox_min = [0,0,0]

In [ ]:
df = pd.read_csv(os.path.join(db_path, 'dataset.csv'))
pc_names = df['pc_name'].unique()
df = df.set_index(['pc_name', 'codec_id', 'codec_rate']).sort_index()

In [ ]:
icip_pcs = []
for idx, data in tqdm(df.iterrows()):
    icip_pcs.append(icip_pc(idx[0],idx[1],idx[2],data['geometry_bits'],data['mos'],data['mos_ci'],data['relative_path'],data['radius']))       
for pc in tqdm(icip_pcs):
    pc.load_points()
    pc.connect_with_ref(icip_pcs)
    pc.partition()
    pc.load_tree()
for pc in tqdm(icip_pcs):
    pc.load_dists_ngbs()
    pc.compute_features()
    pc.find_shared_blocks()

In [ ]:
icip_partitions = {}
for name in pc_names:
    icip_block_names = []
    test_names = []
    for pc in icip_pcs:
        if pc.pc_name != name and pc.is_ref == False :
            for block in pc.shared_blocks:
                icip_block_names.append([pc.id, block])
        if pc.pc_name == name and pc.is_ref == False:
            for block in pc.shared_blocks:
                test_names.append([pc.id, block])
    icip_partitions[name] = {'train' : icip_block_names, 'test' : test_names}

In [ ]:
filters=32
block_shape_modified=(64,64,64,1)
params = {'strides': (2, 2, 2), 'padding': 'same', 'use_bias': True}
Embedding = tf.keras.Sequential()
Embedding.add(tf.keras.layers.Conv3D(name='conv3d_0', filters=32, kernel_size=(5, 5, 5), **params, input_shape=block_shape_modified))
Embedding.add(tf.keras.layers.ReLU())
Embedding.add(tf.keras.layers.Conv3D(name='conv3d_1', filters=32, kernel_size=(5, 5, 5), **params))
Embedding.add(tf.keras.layers.ReLU())
Embedding.add(tf.keras.layers.Conv3D(name='conv3d_2', filters=32, kernel_size=(5, 5, 5), **params))
Embedding.add(tf.keras.layers.ReLU())
Embedding.add(tf.keras.layers.Conv3D(name='conv3d_3', filters=8, kernel_size=(1, 1, 1), activation= tf.keras.activations.relu,strides=(1,1,1)))
Embedding.add(tf.keras.layers.Flatten(name='flatten'))

In [ ]:
right_input = tf.keras.Input((64,64,64,1))
left_input = tf.keras.Input((64,64,64,1))
right_y = Embedding(right_input)
left_y = Embedding(left_input)
b = tf.keras.layers.Concatenate()([right_y, left_y]) 
b = tf.keras.layers.Dropout(rate=0.5)(b)
b=tf.keras.layers.Dense(32,activation='relu')(b)
b = tf.keras.layers.Dropout(rate=0.5)(b)
b=tf.keras.layers.Dense(4,activation='relu')(b)
b=tf.keras.layers.Dense(1,activation='relu')(b)
Siamese = tf.keras.Model(inputs = [right_input, left_input ], outputs = [b], name="siamese")

In [ ]:
def shuffle_and_split(number, train_ratio):
    randomlist = np.arange(number)
    np.random.shuffle(randomlist)
    train_randomlist = randomlist[0:round(number*(1-train_ratio))]
    set_randomlist = set(randomlist)
    set_train_randomlist = set(train_randomlist) 
    validation_randomlist = set_randomlist-set_train_randomlist
    train_names = np.array(list(set_train_randomlist))
    np.random.shuffle(train_names)
    validation_names = np.array(list(validation_randomlist))
    np.random.shuffle(validation_names)
    return train_names, validation_names

In [ ]:
def icip_push_sample (num, name, mode): 
    num = num.numpy()
    name = str(name.numpy(),encoding='ascii')
    mode = str(mode.numpy(),encoding='ascii')
    pc_id = icip_partitions[name][mode][num][0]
    block = icip_partitions[name][mode][num][1]
    for pc in icip_pcs:
        if pc.id == pc_id :
            x1 = pc.blocks_meta[block]['block']
            x2 = pc.ref.blocks_meta[block]['block']
            mos = pc.mos/5
    zeros1 = np.zeros(block_shape, dtype=np.float32)
    x1 = pts_to_vx(x1, block_shape, zeros1)
    x1 = x1.reshape([64,64,64,1])
    zeros2 = np.zeros(block_shape, dtype=np.float32)
    x2 = pts_to_vx(x2, block_shape, zeros2)
    x2 = x2.reshape([64,64,64,1])
    return x1, x2, mos
def tf_icip_push_sample (num, name, mode):
    return tf.py_function(icip_push_sample, [num, name, mode], [tf.float32, tf.float32, tf.float32])
def icip_divide_sample (x1, x2, d2):
    return ((x1, x2), d2)

In [ ]:
def initialize_weights(model, weights=None):
    if weights is None:
        weights = model.get_weights()
    initializer = tf.keras.initializers.HeUniform()
    weights = [initializer(shape = w.shape) for w in weights]
    model.set_weights(weights)

In [ ]:
initial_learning_rate = 1e-03
def lr_exp_decay(epoch, lr):
    k = 0.1
    return initial_learning_rate * tf.math.exp(-k*epoch)

In [ ]:
for pc in icip_pcs: pc.sum_var = 0
    
for name in tqdm(icip_partitions.keys()):
    
    icip_block_names = icip_partitions[name]['train']
    train_index, val_index = shuffle_and_split(len(icip_block_names), 0.1)
    train_dataset = tf.data.Dataset.from_tensor_slices(train_index)#.take(3)
    name_dataset = tf.data.Dataset.from_tensor_slices(np.asarray([name for i in range(len(train_index))]))
    mode_dataset = tf.data.Dataset.from_tensor_slices(np.asarray(['train' for i in range(len(train_index))]))
    icip_dataset = tf.data.Dataset.zip((train_dataset, name_dataset, mode_dataset))
    icip_dataset = icip_dataset.map(tf_icip_push_sample, num_parallel_calls = 64)
    icip_dataset = icip_dataset.map(icip_divide_sample, num_parallel_calls = 64)
    icip_dataset = icip_dataset.batch(64).prefetch(1)
    
    val_dataset = tf.data.Dataset.from_tensor_slices(val_index)
    icip_dataset_val = tf.data.Dataset.zip((val_dataset, name_dataset, mode_dataset))
    icip_dataset_val = icip_dataset_val.map(tf_icip_push_sample, num_parallel_calls = 64)
    icip_dataset_val = icip_dataset_val.map(icip_divide_sample, num_parallel_calls = 64)
    icip_dataset_val = icip_dataset_val.batch(64).prefetch(1)   
    initialize_weights(Siamese)
    callback = tf.keras.callbacks.EarlyStopping(monitor='val_loss', patience=100, restore_best_weights=True)
    reduce_lr = tf.keras.callbacks.ReduceLROnPlateau(monitor='loss', factor=0.1, patience=3)
    lr_scheduler = tf.keras.callbacks.LearningRateScheduler(lr_exp_decay, verbose=0)
    Siamese.trainable = True
    Siamese.compile(optimizer=tf.keras.optimizers.Adam(learning_rate=1e-03), loss=tf.keras.losses.MeanSquaredError()) 
    history = Siamese.fit(icip_dataset, epochs=100, callbacks=[callback,reduce_lr,lr_scheduler], initial_epoch=0 ,validation_data=icip_dataset_val)
    
    Siamese.trainable = False
    Siamese.compile(optimizer=tf.keras.optimizers.Adam(), loss=tf.keras.losses.MeanSquaredError())
    test_names = icip_partitions[name]['test']
    mode_dataset = tf.data.Dataset.from_tensor_slices(np.asarray(['test' for i in range(len(test_names))]))
    test_dataset = tf.data.Dataset.from_tensor_slices(np.arange(len(test_names)))#.take(3)
    icip_dataset_test = tf.data.Dataset.zip((test_dataset, name_dataset, mode_dataset))
    icip_dataset_test = icip_dataset_test.map(tf_icip_push_sample, num_parallel_calls = 128)
    icip_dataset_test = icip_dataset_test.map(icip_divide_sample, num_parallel_calls = 128)
    icip_dataset_test = icip_dataset_test.batch(128).prefetch(1)
    predictions = Siamese.predict(icip_dataset_test)
    icip_partitions[name]['predictions'] = predictions
    
    for i, elem in enumerate(test_names):
        for pc in icip_pcs :
            if pc.id == elem[0] :
                pc.sum_var = pc.sum_var + predictions[i]

In [ ]:
mos_list = np.reshape(np.asarray([pc.mos for pc in icip_pcs if pc.is_ref == False][0:75]), -1)
preidctions_list = np.reshape(np.asarray([pc.sum_var/len(pc.shared_blocks) for pc in icip_pcs if pc.is_ref == False][0:75]), -1)

In [ ]:
plcc=scipy.stats.pearsonr(mos_list, preidctions_list)
srocc=scipy.stats.spearmanr(mos_list, preidctions_list)

In [ ]:
print(plcc)
print(srocc)

(0.8356561950371263, 1.1160330465589438e-20)
SpearmanrResult(correlation=0.8348971058303803, pvalue=1.3018166846360089e-20)
